In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

# Interfaces: Stacks

The `Stack` class was described in the [Anatomy of a simple class](../part2/anatomy_of_a_simple_class.ipynb#notebook_id) notebook. The `Stack` class implemented in that notebook was a data structure representing a stack of strings. The implementation of the class is shown below:

In [ ]:
import java.util.ArrayList;

public class Stack {

    private ArrayList<String> stack;
    
    public Stack() {
        this.stack = new ArrayList<>();
    }

    public int size() {
        return this.stack.size();
    }

    public void push(String elem) {
        this.stack.add(elem);
    }

    public String pop() {
        String elem = this.stack.remove(this.size() - 1);
        return elem;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

The `List`-based implementation of `Stack` uses an `ArrayList` to store the elements of the stack where the end of the list corresponds to the top of the stack. The computational complexity of
`pop` is in $O(1)$ and the computational complexity of `push` is usually in $O(1)$ (but is in $O(n)$ when the array capacity of the list is exceeded).

**Exercise 1** What is complexity of `push` and `pop` if the front of the list corresponds to the top of the stack?

There are at least two main alternative implementations of a `Stack`:

* an array-based implementation
    * the implementer must manage the resizing of the array manually
        * this is a useful exercise for learning the mechanics of array management, but there are no significant performance differences compared to the `List`-based implementation in most applications
* a linked node based implementation
    * the worst-case computational complexity of `pop` and `push` are in $O(1)$ but the typical call to `pop` or `push` will be slower than the `List`-based implementation
    * such an implementation is actually an implementation of a singly linked list
    
The three different stack implementations are all very different, but they share the same method contracts. By creating a Java interface and having each of the different implementations
implement the interface, we gain the ability to use the different types of stacks interchangably.

<div class="alert alert-block alert-warning">
    The concept of a stack is an example of an <i>abstract data type</i>. An abstract data type is a mathematical model of a data type. An abstract data
    type defines the values that the type can have, the operations that can be done with the type, and the results of performing an operation. An abstract
    data type is a theoretical construct; it is independent of any programming language and it <i>does not define the implementation details of the data type</i>.
    Indeed, there may be more than one way to implement the data type in any given programming language, and the different implementations may have
    different strengths and weaknesses.<br /><br />
    A Java interface can be used as a Java specification of an abstract data type. A class that implements the interface is said to be a <i>concrete
    data structure</i> or simply a <i>data structure</i>. 
</div>

## A `Stack` interface

Creating an interface is similar to creating a class with some key differences:

* use the keyword `interface` instead of `class` when declaring the interface
* there are no instance fields (no non-`static` fields)
    * only `public static final` fields are allowed
        * if is a compile-time error if the access modifier `private` or `protected` is used in a field declaration
        * if the `static` or `final` modifier is missing from a field declaration then the compiler adds the missing modifiers
* the methods of an interface are all `public`
    * if the access modifier is missing then `public` access is assumed
        * it is a compile-time error to specify a non-`public` access modifier
* most methods have no body
    * the method header ends with a semi-colon
* `default` methods do have a body
    * `default` methods can use other methods defined in the interface (and other classes and interfaces)
    
A `Stack` interface is shown in the following cell:

In [ ]:
/**
 * The {@code Stack} interface represents a last-in-first-out (LIFO) stack of
 * strings. In addition to the usual push and pop methods, this interface
 * allows the user to get the number of strings in a stack and to query
 * if the stack is empty.
 */
public interface Stack {

    /**
     * Returns the number of elements in this stack.
     * 
     * @return the number of elements in this stack
     */
    public int size();

    /**
     * Returns {@code true} if this stack contains no elements. The default
     * implementation simply returns {@code size() == 0}.
     * 
     * @return true if this stack contains no elements
     */
    default public boolean isEmpty() {
        return this.size() == 0;
    }

    /**
     * Pushes the specified element on to the top of this stack.
     * 
     * @param elem the element to be pushed on to the top of this stack
     */
    public void push(String elem);

    /**
     * Removes the element on the top of this stack and returns the element.
     * 
     * @return the top element of this stack
     * @throws RuntimeException if the stack is empty
     */
    public String pop();
}

Notice that interface has the `default` method `isEmpty` that returns `true` if the stack has zero elements. The `isEmpty` method is a *convenience method*: It is non-essential because
the caller can test the return value from the `size` method to achieve the same result, but it is much more convenient for the caller to use. Because `isEmpty` can be implemented
in terms of the other methods in the interface, we can provide the implementation in the interface but we must declare the method to be a `default` method if the implementation
is provided inside the interface.

Also notice that the contract of the `pop` method has been updated so that `pop` throws an exception if an empty stack is popped.

## Implementing an interface

Creating `Stack` objects requires a class that implements the `Stack` interface. We rename our previous class to `ListStack` to reflect the fact that it implements a `List`-based stack
and we add `implements Stack` to its class declaration:

In [ ]:
import java.util.ArrayList;

public class ListStack implements Stack {      // 1, 2

    private ArrayList<String> stack;
    
    public ListStack() {                       // 3
        this.stack = new ArrayList<>();
    }

    @Override                                  // 4
    public int size() {
        return this.stack.size();
    }

    @Override                                  // 5
    public void push(String elem) {
        this.stack.add(elem);
    }

    @Override                                  // 6
    public String pop() {
        if (this.isEmpty()) {                  // added to satisfy the updated contract of pop
            throw new RuntimeException("popped an empty stack");
        }
        String elem = this.stack.remove(this.size() - 1);
        return elem;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

Converting our previous `Stack` class into a class that implements our `Stack` interface requires six minor changes (marked with comments above):

1. the class is renamed to `ListStack` because we cannot have two different types with the same name in the same package
    * i.e., we cannot write `public class Stack implements Stack`
2. we add `implements Stack` after the class name to indicate that the class implements the `Stack` interface
3. we change the constructor name to `ListStack` so that it matches the new name of the class
4. we add the *annotation tag* `@Override` before the `size` method
5. we add the annotation tag `@Override` before the `push` method
6. we add the annotation tag `@Override` before the `pop` method

Changes 4-6 are optional but strongly recommended: A class that implements an interface should annotate the interface methods with an `@Override` tag.

We've seen the `@Override` tag in Part 2 of the notes when we studied overriding the `toString`, `equals`, and `hashCode` methods. The `@Override` tag forces the compiler to check if the
programmer has used the correct method modifiers, return type, and method signature when overriding a method. When implementing an interface, the `@Override` tag forces the compiler to check if the
programmer has used the same method modifiers, return type, and method signature that were used in the interface.

<div class="alert alert-block alert-warning">
    <b>Why is the method <tt>isEmpty</tt> not in <tt>ListStack</tt>?</b><br />
    <tt>ListStack</tt> does not need to implement the <tt>isEmpty</tt> method because the <tt>isEmpty</tt> is a default method that already has an implementation 
    in the <tt>Stack</tt> interface. We say that the <tt>ListStack</tt> class <i>inherits</i> the implementation of
    <tt>isEmpty</tt> from the interface.
</div>

**Exercise 2** Does the `Stack` interface still compile if you remove the `public` access modifier from the `Stack` interface declaration?

**Exercise 3** Does the `Stack` interface still compile if you remove the `public` access modifier from the methods declared/defined in the `Stack` interface?

**Exercise 4** Can you change the `public` access modifier to some other access modifier for the `Stack` methods implemented in the `ListStack` class?

**Exercise 5** Can you implement a version of `toString` as a `default` method in the `Stack` interface. Try to do so. Your method should include a
string representation of all of the elements in the stack *and* it should not permanently alter the state of the stack.